# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    23-Jan-2026**

**News Source / Catalyst:** "Bank of Japan warns over rapid rise in bond yields as Takaichi dissolves parliament for snap poll"

**Markets (s) Impacted:** FX (JPY), EQ (Banks vs Retail), FI (JGB Yields)

**Trade Taxonomy:** RV Sector (Monetary-Fiscal Divergence)

**Trade Idea (1 sentence):**
> Long the BOJ rate-hike beneficiares (Mega Banks) while shorting the domestic-focused retailers vulnerable to fiscal noise and currency volatility.

**Client/Desk Pitch (30 seconds):**
> Japan is entering a rare period of "Fiscal-Monetary Friction". While the PM is launching a JPY 21tn *fiscal dove* stimulus and a 16-day snap election campaign, the BOJN is moving as a *monetary hawk* with revised 1.9% inflation forecasts. This divergence spikes JGB yields to 2.25% (a 27y high). We are going Long Japanese banks to capture widening NIMs and Short Retailers who face a margin squeexe from import costs and election-driven consumption uncertainty.

**Why Now:**
- **BOJ Confidence**: An 8-1 vote with a call for an immediate hike to 1.0% signals a definitive end to the low rate era
- **Election Window**: The shortest-ever campaign period (23-Jan to 08-Feb) creates a tactical vacuum for high convition relative value
- **Yield Breakout**: 10y JGB yields hitting 2.25% forces an immediate repricing of fiscal vs. industrial sectors

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- **Hawkish Momentum**: BOJ is behind the curve and will likely hike to 1% by April to catch rising underlying inflation
- **Fiscal Excess**: Takaichi's consumption tax suspension plans increase sovereign risk, pushing yields higher than central bank targets
- **Margin Divergence**: Banks gain from yield curve steepening; domestic retailers suffer from "cost-push" inflation and JGB-driven yen volatilty

**Long Leg (Tickers + Rationale):**
- 8306.T (MUFG), 8316.T (SMFG): Direct beneficiaries of the shift from 0.75% to 1.0% rates; expanding Net Interest Margins

**Short Leg (Tickers + Rationale):**
- 9983.T (Fast Retailing), 3382.T (Seven & I): High sensitivity to Yen-driven import costs and election-period consumption pauses.

**Weights:** Beta-neutral (Vol-adjusted)

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |  S                | JPY volatility is the trade's primary noise source       |
| Oil Beta             |  W                | Minimal direct impact compared to rates      |
| Rates Duration       |  S                | Strategy is a pure-play on 10y JGB yield steepening      |
| Growth vs Value      |  S                | Classic Value (Banks) vs Growth/Consumption (Retail) spread     |
| Risk-On/Off          |  M                | Election uncertainty can trigger a broader JPY risk-off bid      |
| China/EM Beta        |  W                | Domestically focused Japanese policy trade       |
| Volatility Regime    |  S                | Profiting from the transition to a high-rate regime      |

**Catalyst Type:** Regime Shift (End of Zero-Bound Japan)

**Realisation Window:** Weeks (Election Campaign window)

**Holding Period Selected:** 16 Days (Targeting 06-Feb exit)  

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** (units per leg)  Beta-weighted (approx. 1.4x notional in Banks vs 1x Retailers)

**Stop-Loss:** -2.5% portfolio-level

**Take-Profit (optional):**  4.0% portfolio level

**Re-entry:** No (Election trades are one-off event windows)

**FX-Hedging Required:** Yes (if trading via USD-denominated accounts)

**Liquidity Notes:** ADV in MUFG/9983.T is extremely high; slippage is negligible

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information (Central Bank yield curve control intervention)

---


# **PYTHON IMPLEMENTATION**

In [12]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [13]:
# --- Parameters ---

long_tickers = ["8306.T", "8316.T"]  # MUFG and SMFG (Banks)
short_tickers = ["9983.T", "3382.T"] # Fast Retailing and Seven & I (Retail)

start_date = "2024-01-01"                         # Backtest window
holding_period = 16                               # days
stop_loss = -0.025                                 # -2% stop
take_profit = 0.04                                # optional e.g. +0.03 for +3%

# --- Weightings ---

weights_long = 0.7  # (e.g., 1.4x multiplier for Banks lower vol)
weights_short = 0.5 # (Standard weight for Retailers)


In [14]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_53861/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  4 of 4 completed


In [15]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = (long_returns * weights_long) - (short_returns * weights_short)

In [16]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
26,2025-09-17,0.0477
27,2025-10-10,-0.0282
28,2025-11-05,0.0406
29,2025-11-28,0.0444
30,2025-12-22,0.0423


In [17]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         31.0000
Win Rate              0.7742
Average PnL           0.0209
Best Trade            0.0659
Worst Trade          -0.0542
Stop-Out Frequency    0.1613
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 31.0000
- Win Rate: 0.7742
- Average PnL: 0.0209
- Best Trade: 0.0659
- Worst Trade: -0.0542
- Stop-Out Frequency: 0.1613

**What went right:**
- **High Convexity**: The win rate of 77% suggests a very strong "Regime Shift" signal. The Banks (MUFG/SMFG) are capturing the non-linear repricing of the 10Y JGB yield effectively, while the short retail leg provides a consistent buffer against broad market volatility
- **Positive Drift**: An average PnL of 2.09% over a 16-day window is exceptional for a market-neutral strategy, indicating the divergence between monetary tightening and fiscal expansion is widening faster than the market can arbitrage it.


**What went wrong:**
- **Fat Tail Risk**: The worst trade (-5.42%) exceeded the portfolio stop-loss setting significantly. This indicates "gap risk", likely an overnight announcement of emergency BoJ bond-buying or an FX intervention that caused a violent reversal in JGB yields before the trade could be liquidated
- **Stop-Out Sensitivity**: A 16% stop-out frequency shows that while the thesis is correct most of the time, the trade is highly sensitive to verbal intervention from the Ministry of Finance

**Factor that actually drove results:**
- **JGB Yield Steepening**: The spread is essentially a proxy for the 10Y-2Y yield curve gap. As Takaichi’s deficit spending plans push the long end of the curve up (2.25%) and the BoJ anchors the short end (0.75%), the "carry" and "NIM expansion" for banks becomes the dominant P&L driver

**Would I trade this live?** (Yes / No and why)
> Yes. A win rate near 80% with a positive expectancy of 2%+ per trade is a high-conviction setup. However, I would tighten the execution logic to include a "trailing stop" once PnL hits 3% to protect against the "Information Failure Mode" (BoJ emergency intervention) that caused the -5.42% outlier.

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [18]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [19]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [20]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
26,RV-E05E95BD,2026-01-23 10:55:32,Macro RV Spread,RV Macro Spread,"8306.T, 8316.T","9983.T, 3382.T",2025-09-17,2025-10-09,-20422.6027,-20614.4000,1.8781,-2.5000,4.0000,CLOSED,22,
27,RV-6B994028,2026-01-23 10:55:32,Macro RV Spread,RV Macro Spread,"8306.T, 8316.T","9983.T, 3382.T",2025-10-10,2025-10-10,-22312.3000,-22312.3000,-2.8168,-2.5000,4.0000,STOPPED,0,
28,RV-60B5E508,2026-01-23 10:55:32,Macro RV Spread,RV Macro Spread,"8306.T, 8316.T","9983.T, 3382.T",2025-11-05,2025-11-27,-24573.0000,-24737.2500,2.0298,-2.5000,4.0000,CLOSED,22,
29,RV-DFAF3245,2026-01-23 10:55:32,Macro RV Spread,RV Macro Spread,"8306.T, 8316.T","9983.T, 3382.T",2025-11-28,2025-12-19,-24658.9500,-24057.1000,2.4911,-2.5000,4.0000,CLOSED,21,
30,RV-84954DA8,2026-01-23 10:55:32,Macro RV Spread,RV Macro Spread,"8306.T, 8316.T","9983.T, 3382.T",2025-12-22,2026-01-16,-24522.2000,-26393.8000,8.8024,-2.5000,4.0000,TAKE_PROFIT,25,


In [21]:
blotter.to_csv("trade_blotter.csv", index=False)